In [1]:
import os
import pandas as pd
import datetime as dt
import numpy as np
import re

In [2]:
column_names = {'Emiasid': 'emiasid', 'Emiasid ': 'emiasid' , 'emiasid ': 'emiasid', 'EMIAS_ID': 'emiasid', 'МО': 'mo', 
                'МУ': 'mu', 'Логин оператора': 'login', 'Дата и время звонка': 'call_date', 'Результат звонка': 'call_result',
                'Результат звонка ': 'call_result', 'Разговор состоялся': 'talk_result', 'Комменатрий': 'comment', 
                'Дата и время переноса': 'next_call_date', 'Контактный телефон': 'phone_number', 'ФИО врача': 'doctor_fio', 
                'ФИО помощника врача': 'assistant_fio', 'Логин помощника врача': 'login', 'Зал №': 'hall_number',
                'Логин naumen': 'login', 'МО закрепления': 'mo', 'Отсутствует на рабочем месте': 'absence'}

In [3]:
today = pd.Timestamp(dt.datetime.today().date())
today_ymd = today.strftime('%Y%m%d')
today_y_m_d = today.strftime('%Y_%m_%d')

yesterday = today - dt.timedelta(days=1)
yesterday_ymd = yesterday.strftime('%Y%m%d')
yesterday_y_m_d = yesterday.strftime('%Y-%m-%d')

tomorrow = today + dt.timedelta(days=1)
tomorrow_ymd = tomorrow.strftime('%Y%m%d')
tomorrow_y_m_d = tomorrow.strftime('%Y-%m-%d')

print(today_y_m_d)

2022_09_14


In [4]:
# путь файла с текущими статусами пациентов
patients_current_statuses_path = \
r'\Users\murad_satabaev\Documents\GitHub\final_framework\merged_statuses'

patients_current_statuses_path = patients_current_statuses_path.split('\\')
patients_current_statuses_data = os.path.join('C:\\', r'\\', *patients_current_statuses_path,
                                             f'total_patients_current_statuses_merged_with_logins_{today_y_m_d}.xlsx')
print(patients_current_statuses_data)

C:\\Users\murad_satabaev\Documents\GitHub\final_framework\merged_statuses\total_patients_current_statuses_merged_with_logins_2022_09_14.xlsx


In [5]:
# путь файла с сегодняшними 
emias_data_path = r'\\t999\Сетевой диск\pdn\data\emias_data\xlsx'
emias_data_path = emias_data_path.split('\\')
emias_data_path = os.path.join(r'\\', *emias_data_path)
print(emias_data_path)

\\t999\Сетевой диск\pdn\data\emias_data\xlsx


In [6]:
today_emias_data_path = [os.path.join(emias_data_path, filename) for filename in os.listdir(emias_data_path) if '4059' 
                         in filename and '~' not in filename and today.strftime('%Y-%m-%d') in filename][0]
print(today_emias_data_path)

\\t999\Сетевой диск\pdn\data\emias_data\xlsx\DAT_4059_выгрузка_списков_на_обзвон_помощниками_2022-09-14.xlsx


In [7]:
# чтение всех файлов по отдельности, чтобы не тратить время на повторное чтение файла, который уже прочитан

In [8]:
today_emias_data_reading = pd.read_excel(today_emias_data_path)
today_emias_data_reading.shape[0]

97343

In [9]:
patients_current_statuses_reading = pd.read_excel(patients_current_statuses_data)
patients_current_statuses_reading.shape[0]

109888

In [10]:
stop()

NameError: name 'stop' is not defined

In [13]:
import gspread
from gspread_dataframe import get_as_dataframe
gc = gspread.service_account(filename='google_sheets_credentials.json') 
url = 'https://docs.google.com/spreadsheets/d/1TXjbk9tRDMCjEyyiuK5OIgT8xwlHQ2V1PX2zaVnxPXY/edit#gid=0'
sh = gc.open_by_url(url)
ws = sh.get_worksheet(0)
def search_row(ws, desired_value):
    row_number = 0
    for row in ws.get_all_values():
        row_number += 1
        for value in row:
            if value == desired_value:
                return row_number
            
skiprows = search_row(ws, 'ФИО помощника врача') - 1

assistants_presence_reading = get_as_dataframe(ws, evaluate_formulas=True, skiprows=skiprows)
assistants_presence_reading = assistants_presence_reading.loc[~(assistants_presence_reading['Зал №'] == 'Зал №'), 
    ['Зал №', 'МО закрепления', 'ФИО помощника врача', 'Логин naumen', 'Отсутствует на рабочем месте']] \
        .dropna(how='all', axis=0)


print(dt.datetime.now())
print(assistants_presence_reading.shape[0])

assistants_presence = assistants_presence_reading.rename(columns=column_names).copy()

2022-09-14 18:20:28.757167
217


In [ ]:
stop()

In [14]:
patients_current_statuses = patients_current_statuses_reading.rename(columns=column_names).copy()

In [15]:
today_emias_data = today_emias_data_reading.rename(columns=column_names)

In [16]:
patients_current_statuses = patients_current_statuses_reading.rename(columns=column_names).copy()
patients_current_statuses['emiasid'] = patients_current_statuses['emiasid'].astype(str).str.replace('\.0', '')
patients_current_statuses['next_call_date'] = patients_current_statuses['next_call_date'] \
.apply(lambda x: pd.to_datetime(x, dayfirst=True).date())
patients_current_statuses['last_call_date'] = patients_current_statuses['last_call_date'] \
.apply(lambda x: pd.to_datetime(x, dayfirst=True).date())

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_4916\1251941699.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  patients_current_statuses['emiasid'] = patients_current_statuses['emiasid'].astype(str).str.replace('\.0', '')


In [ ]:
# Переведем все типы данных емиасайди, телефоны и прочие числовые данные в строки

In [17]:
today_emias_data['emiasid'] = today_emias_data['emiasid'].astype(str).str.replace('\.0', '')
today_emias_data['phone'] = today_emias_data['phone'].astype(str).str.replace('\.0', '')

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_4916\3971369908.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  today_emias_data['emiasid'] = today_emias_data['emiasid'].astype(str).str.replace('\.0', '')
C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_4916\3971369908.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  today_emias_data['phone'] = today_emias_data['phone'].astype(str).str.replace('\.0', '')


In [18]:
def find_phone_number(text):
    """Функция ищет номера телефонов в строке, фильтрует дефектные и возвращает один с приоритетом на мобильный"""
    import re
    defect_pattern = re.compile(r"""7[^94]\d{9}|0{7,}|1{7,}|2{7,}|3{7,}|4{7,}|5{7,}|6{7,}|7{7,}|8{7,}|9{7,}""")
    text = str(text)
    text = ['7' + re.sub(r'\D', '', x)[-10:] for x in re.split(r'[,;]', text)]
    text = list(filter(lambda x: len(x) == 11, text))
    text = list(filter(lambda x: False if re.search(defect_pattern, x) else True, text))
    text = sorted(text, reverse=True)
    text = ','.join(text)
    text = re.search(r'(9\d{9}|4\d{9})', text)
    if text:
        text = text.group()
    return text

today_emias_data['phone_1'] = today_emias_data['phone'].apply(find_phone_number)
today_emias_data[today_emias_data['phone_1'].isna()].shape[0]

876

In [19]:
today_emias_data.shape[0]

97343

In [20]:
patients_current_statuses.shape[0]

109888

In [21]:
actual_patients = \
patients_current_statuses[
    (patients_current_statuses.problem.isna())
]

In [22]:
patients_current_statuses.shape[0]

109888

In [23]:
actual_patients.shape[0]

107478

In [ ]:
# # уберем пациентов без прикрепления к врачу и тех, кого нет в сегодняшней выгрузке
# patients_current_statuses = \
# patients_current_statuses[patients_current_statuses.emiasid.isin(
# today_emias_data[
#     ~((today_emias_data.doctor_fio.isna())
#      | 
#      (today_emias_data.phone_1.isna())
#     )
# ].emiasid
    
# )]
# print(patients_current_statuses.shape[0])

In [ ]:
stop()

In [ ]:
# if assistant_to_doctor_distribution[assistant_to_doctor_distribution.login.isin(assistants_presence.login)].login.nunique() != \
# assistants_presence[assistants_presence.login.isin(assistants_presence.login)].login.nunique():
#     print(5 * '\nКОЛИЧЕСТВО ПОМОЩНИКОВ В РАСКРЕПЛЕНИИ И "НА СМЕНЕ" РАЗНОЕ !!!\n!!!\n')
# else:
#     print('\nКоличество помощников в раскреплении "на смене" соответсвтует\n')

In [24]:
if assistants_presence[assistants_presence.absence.isna()].login.nunique() + \
assistants_presence[assistants_presence.absence == 1].login.nunique() == assistants_presence.login.nunique():
    print('\nВ сумме количество отсуствующих и присутствующих равны общему количеству помощников\n')
else:
    print(5 * '\nПРОВЕРЬТЕ ТАБЛИЦУ ОТСУТСТВУЮЩИХ И ПРИСУТСТВУЮЩИХ\n!!!\n')


В сумме количество отсуствующих и присутствующих равны общему количеству помощников



In [25]:
present_assistants = assistants_presence[assistants_presence.absence.isna()]
print(present_assistants.shape[0])
absent_assistants = assistants_presence[~assistants_presence.absence.isna()]
print(absent_assistants.shape[0])

166
51


In [ ]:
doctors_list = set(assistant_to_doctor_distribution.doctor_fio.unique())
doctors_list.add(np.NaN)

In [ ]:
stop()

In [ ]:
# actual_patients_for_task = patients_current_statuses[patients_current_statuses.group.isin({
#     11., 25., 22., 19.,  3., 17.,  9.,  6., 14., 21.,  8.,    15., 
#     5., 23., 13., 20., 16., 18.,  4., 12.,  7.,  2.,     10. })].copy()
# actual_patients_for_task.shape[0]

In [26]:
first_stage_patients = \
actual_patients[actual_patients.status == 'Ни разу не звонили']

first_stage_patients['priority'] = 2
first_stage_patients.shape[0]

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_4916\3421971818.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_stage_patients['priority'] = 2


9515

In [29]:
call_back_on_date_patients = \
actual_patients[
    (actual_patients.last_call_result == 'Перенос звонка на другой день')
    &
    (actual_patients.next_call_date.isin({
        dt.date(2022, 9, 15)
    }))
]

call_back_on_date_patients['priority'] = 1
call_back_on_date_patients.shape[0]

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_4916\3568402929.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  call_back_on_date_patients['priority'] = 1


2947

In [30]:
second_stage = pd.read_excel('2_цикл_емиасайди.xlsx')
second_stage['emiasid'] = second_stage['emiasid'].astype(str).str.replace('\.0', '')
second_stage.shape[0]

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_4916\1002607819.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  second_stage['emiasid'] = second_stage['emiasid'].astype(str).str.replace('\.0', '')


6769

In [31]:
august_patients = \
actual_patients[
    (actual_patients.month == 'august')
    &
    (actual_patients.last_call_result != 'Перенос звонка на другой день')
    &
    (~(actual_patients.emiasid.isin(second_stage.emiasid)))
    &
    (actual_patients.last_call_date < dt.date(2022, 9, 2))
]

august_patients['priority'] = 3
august_patients.shape[0]

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_4916\671480930.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  august_patients['priority'] = 3


3033

In [32]:
august_patients.last_call_date.unique()[-1]

datetime.date(2022, 9, 1)

In [33]:
tomorrow_record_patients_set = set()
for column in today_emias_data.columns[22:98:2]:
    tomorrow_record_patients_set = \
    tomorrow_record_patients_set | set(today_emias_data[today_emias_data[f'{column}'] == tomorrow_y_m_d]\
                                   .emiasid.astype(str).unique())

print(len(tomorrow_record_patients_set))

4150


In [34]:
tomorrow_record_patients = patients_current_statuses[(patients_current_statuses.emiasid.isin(tomorrow_record_patients_set))
& (~(patients_current_statuses.status.isin({'Ни разу не звонили'}))) & 
(patients_current_statuses.last_call_result != 'Перенос звонка на другой день')]
print(tomorrow_record_patients.shape[0])

1866


In [35]:
tomorrow_record_patients['priority'] = 5

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_4916\4186667993.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tomorrow_record_patients['priority'] = 5


In [36]:
tomorrow_record_patients['status'] = 'Напомнить'

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_4916\672541353.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tomorrow_record_patients['status'] = 'Напомнить'


In [37]:
second_stage_patients = \
actual_patients[
    (actual_patients.status.isin({'Госпитализирован', 'Разговор состоялся', 'Не дозвонились', 'Отказ', 
                                'Запишется самостоятельно', 'Не нравится МО', 'Не нравится врач'}))
    &
    (~(actual_patients.emiasid.isin(call_back_on_date_patients.emiasid)))
]

second_stage_patients['priority'] = 4

second_stage_patients.shape[0]

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_4916\1392356965.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_stage_patients['priority'] = 4


50717

In [38]:
final_actual_patients_for_task_ = pd.concat([
    call_back_on_date_patients,
    first_stage_patients,
    august_patients,
#     second_stage_patients,
#     tomorrow_record_patients
]) \
.sort_values(['priority']).reset_index(drop=True)

final_actual_patients_for_task_.shape[0]

15495

In [39]:
final_actual_patients_for_task = final_actual_patients_for_task_.drop_duplicates(subset=['phone_number'], keep='first')
final_actual_patients_for_task.shape[0]

15407

In [40]:
final_actual_patients_for_task = final_actual_patients_for_task.drop_duplicates(subset=['emiasid'], keep='first')
final_actual_patients_for_task.shape[0]

15407

In [41]:
final_actual_patients_for_task.head(3)

,emiasid,operator_login,status,old_comment,next_call_date,mo,doctor_fio,phone_number,calls_amount,last_call_date,last_call_result,last_talk_result,problem,month,group,operator_absence,priority
0,17199307,pomoshnik_pdn80,Будет в Москве в другом месяце,перезвонить,2022-09-15,ГП 64,Джан Екатерина Николаевна,9.776304e+09,2.0,2022-09-13,Перенос звонка на другой день,NaN,NaN,august,NaN,NaN,1
1,25987622,pomoshnik_pdn7,Перенос звонка на другой день,"перезвонить, уточнить записался ли",2022-09-15,ГП 134,Панов Алексей Николаевич,9.151821e+09,1.0,2022-09-08,Перенос звонка на другой день,NaN,NaN,september,11.0,NaN,1
2,23899870,pomoshnik_pdn85,Перенос звонка на другой день,"перезвонить, сз",2022-09-15,ГП 210,Дикаева Макка Султановна,9.252174e+09,1.0,2022-09-08,Перенос звонка на другой день,NaN,NaN,september,11.0,NaN,1


In [42]:
final_actual_patients_for_task = \
final_actual_patients_for_task[['emiasid', 'status', 'old_comment', 'operator_login', 'mo', 'priority', 'calls_amount',
                                'last_call_date', 'month']]

In [43]:
final_actual_patients_for_task['emiasid'] = final_actual_patients_for_task['emiasid'].astype(str).str.replace('\.0', '')

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_4916\2998909243.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  final_actual_patients_for_task['emiasid'] = final_actual_patients_for_task['emiasid'].astype(str).str.replace('\.0', '')


In [44]:
initial_assistant_to_patient_distribution_in_today_task = \
final_actual_patients_for_task.copy()

In [45]:
initial_assistant_to_patient_distribution_in_today_task.old_comment.nunique()

1718

In [46]:
initial_assistant_to_patient_distribution_in_today_task.groupby(['mo', 'operator_login']).agg({'emiasid': 'count'}).sort_values(
    ['mo', 'emiasid']).head(20)

emiasid
mo     operator_login           
ГП 107 pomoshnik_pdn148       14
       pomoshnik_pdn43        24
       pomoshnik_pdn131       26
       pomoshnik_pdn129       28
       pomoshnik_pdn130       31
       pomoshnik_pdn109       45
       pomoshnik_pdn185       55
ГП 109 pomoshnik_pdn134        9
       pomoshnik_pdn155       21
       pomoshnik_pdn168       39
ГП 11  pomoshnik_pdn32        33
       pomoshnik_pdn147       40
       pomoshnik_pdn184       69
       pomoshnik_pdn146      350
ГП 134 pomoshnik_pdn63        17
       pomoshnik_pdn93        17
       pomoshnik_pdn89        43
       pomoshnik_pdn75        70
       pomoshnik_pdn7         94
ГП 166 pomoshnik_pdn209       58

In [ ]:
# initial_assistant_to_patient_distribution_in_today_task.to_excel('initial_assistant_to_patient_distribution_in_today_task.xlsx')

In [83]:
task_amount = 300

In [84]:
%%time

initial_task_for_today = pd.DataFrame()
for operator_login in set(initial_assistant_to_patient_distribution_in_today_task.operator_login):
    initial_assistant_to_patient_distribution_in_today_task_for_operator_login = \
    initial_assistant_to_patient_distribution_in_today_task[initial_assistant_to_patient_distribution_in_today_task \
                                                            .operator_login
         == operator_login].sort_values(by=['priority', 'calls_amount', 'last_call_date'])
    call_back_number_for_operator_login = initial_assistant_to_patient_distribution_in_today_task_for_operator_login[
        initial_assistant_to_patient_distribution_in_today_task_for_operator_login.status.isin({
            'Перезвонить сегодня', 'Перенос звонка на другой день'})].shape[0]

    initial_task_for_today = pd.concat(
        [initial_task_for_today, 

         
         initial_assistant_to_patient_distribution_in_today_task[
             (initial_assistant_to_patient_distribution_in_today_task.operator_login==operator_login)
         ].head(task_amount),
        
       
        ]).reset_index(drop=True)

initial_task_for_today.shape[0]

CPU times: total: 1.55 s
Wall time: 1.58 s


12477

In [85]:
initial_task_for_today \
.groupby(['mo', 'operator_login']).agg({'emiasid': 'count'}).sort_values(['mo', 'emiasid'])\
.sort_values(by='emiasid', ascending=True).head(10)

,,emiasid
mo,operator_login,
ГП 170,pomoshnik_pdn151,1
ГП 69,pomoshnik_pdn264,1
ГП 19,pomoshnik_pdn246,1
ГП 218,pomoshnik_pdn69,1
ГП 69,pomoshnik_pdn263,3
ГП 45,pomoshnik_pdn82,3
ГП 19,pomoshnik_pdn202,3
ГП 52,pomoshnik_pdn103,3
ГП 5,pomoshnik_pdn282,3


In [86]:
initial_task_for_today \
.groupby(['mo', 'operator_login']).agg({'emiasid': 'count'}).sort_values(['mo', 'emiasid'])\
.sort_values(by='emiasid', ascending=False).head(10)

emiasid
mo     operator_login           
ГП 166 Помощник 8            300
ГП 69  pomoshnik_pdn216      300
ГП 191 pomoshnik_pdn52       300
       pomoshnik_pdn15       300
       pomoshnik_pdn13       300
ГП 214 pomoshnik_pdn156      300
       pomoshnik_pdn65       300
ДЦ 3   pomoshnik_pdn100      300
ГП 23  pomoshnik_pdn10       300
       pomoshnik_pdn11       300

In [87]:
rename_mo = \
{'ГБУЗ Г МОСКВЫ ГП № 170 ДЗМ': 'ГП 170', ' ГП 6 ДЗМ': 'ГП 6', 'ГБУЗ КДЦ 6 ДЗМ':'КДЦ 6', 
'ГБУЗ "ГП № 134 ДЗМ"': 'ГП 134', 'КДЦ 4': 'КДЦ 4', 'ГБУЗ ДКЦ 1 ДЗМ': 'ДКЦ 1', 'ГБУЗ ГП 23 ДЗМ': 'ГП 23',
'ГБУЗ ГП 109 ДЗМ': 'ГП 109', 'ГБУЗ ГП 11 ДЗМ': 'ГП 11', 'ГБУЗ ГП 64 ДЗМ': 'ГП 64', 'ГБУЗ ГП 191 ДЗМ': 'ГП 191', 
'ГБУЗ ГП 220 ДЗМ': 'ГП 220', 'ГБУЗ "ГП № 218 ДЗМ"': 'ГП 218',  'ГБУЗ ГП 107 ДЗМ': 'ГП 107', 
'ГБУЗ "ГП № 22 ДЗМ"': 'ГП 22', 'ГБУЗ ДЦ 5 ДЗМ': 'ДЦ 5', 'ГБУЗ ГП 210 ДЗМ': 'ГП 210', 'ГБУЗ ГП 214 ДЗМ': 'ГП 214', 
'ГБУЗ ГП 2 ДЗМ': 'ГП 2', 'ГБУЗ КДП 121 ДЗМ': 'КДП 121', 'ГБУЗ ГП 66 ДЗМ': 'ГП 66',  'ГБУЗ ДЦ 3 ДЗМ ': 'ДЦ 3',
'ГБУЗ ГП 36 ДЗМ': 'ГП 36',  'ГБУЗ ГП 62 ДЗМ ': 'ГП 62', 
'ГБУЗ "ГП № 52 ДЗМ"': 'ГП 52', 'ГБУЗ "ГП 67 ДЗМ"': 'ГП 67', 'ГБУЗ ГП 68 ДЗМ': 'ГП 68', 'ГБУЗ ГП 175 ДЗМ': 'ГП 175',
'ГБУЗ ГП № 209 ДЗМ': 'ГП 209',  'ГБУЗ ГП 212 ДЗМ': 'ГП 212', 'ГБУЗ ГП 19 ДЗМ': 'ГП 19', 
'ГБУЗ ГП №166 ДЗМ': 'ГП 166', 'ГБУЗ ГП 9 ДЗМ': 'ГП 9', 'ГБУЗ ГП 69 ДЗМ': 'ГП 69',
'ГБУЗ "ГП № 3 ДЗМ"': 'ГП 3', 'ГБУЗ ГП 5 ДЗМ': 'ГП 5', 'ГБУЗ ГП 46 ДЗМ': 'ГП 46', 'ГБУЗ ГП №8 ДЗМ': 'ГП 8', 
'ГБУЗ "ГП № 45 ДЗМ"': 'ГП 45'}

In [88]:
mo_list = set(rename_mo.values())

In [89]:
# new_doctor_patients = \
# initial_assistant_to_patient_distribution_in_today_task[(initial_assistant_to_patient_distribution_in_today_task.mo.isin(mo_list)) & 
#                     (~initial_assistant_to_patient_distribution_in_today_task.doctor_fio.isin(doctors_list))].reset_index(drop=True)
# new_doctor_patients = new_doctor_patients[~new_doctor_patients.emiasid.isin(calls_data.emiasid)]

In [90]:
initial_task_for_today_present_assistants = \
initial_task_for_today[~initial_task_for_today.operator_login.isin(absent_assistants.login)]

In [91]:
initial_task_for_today[initial_task_for_today.operator_login.isin(absent_assistants.login)].shape[0]

5733

In [92]:
initial_task_for_today_present_assistants.shape[0]

6744

In [93]:
initial_task_for_today_present_assistants.groupby('operator_login').agg({'emiasid': 'count'}).sort_values('emiasid', ascending=False)

,emiasid
operator_login,
pomoshnik_pdn100,300
pomoshnik_pdn65,300
pomoshnik_pdn307,300
pomoshnik_pdn83,300
pomoshnik_pdn52,300
...,...
pomoshnik_pdn82,3
pomoshnik_pdn202,3
pomoshnik_pdn263,3


In [94]:
initial_task_for_today_absent_assistants =  \
initial_task_for_today[initial_task_for_today.operator_login.isin(absent_assistants.login)] \
[['emiasid', 'status', 'old_comment', 'mo', 'priority', 'month']]

In [95]:
additional_task_for_present_assistants = \
pd.concat([
#     new_doctor_patients,
    initial_task_for_today_absent_assistants
]).reset_index(drop=True)

In [96]:
additional_task_for_present_assistants_by_mo = \
additional_task_for_present_assistants.groupby(['mo']).agg({'emiasid': 'count'}).sort_values(by=['mo', 'emiasid'])

In [97]:
distribution_count = initial_task_for_today_present_assistants.groupby(['mo']) \
.agg({'emiasid': 'count', 'operator_login': 'nunique'}) \
.sort_values(by=['mo', 'emiasid'])

In [98]:
distribution_count['total_delta'] = distribution_count['operator_login'] * task_amount - distribution_count['emiasid']

In [99]:
distribution_count = distribution_count[distribution_count.total_delta > 0]

In [100]:
additional_task_for_present_assistants

,emiasid,status,old_comment,mo,priority,month
0,20884149,Перенос звонка на другой день,перезвонить,ГП 3,1,september
1,22055811,Ни разу не звонили,NaN,ГП 3,2,september
2,22054155,Ни разу не звонили,NaN,ГП 3,2,september
3,22024774,Ни разу не звонили,NaN,ГП 3,2,september
4,22147020,Ни разу не звонили,NaN,ГП 3,2,september
...,...,...,...,...,...,...
5728,16286987,Перенос звонка на другой день,"нет слотов, 8519",ГП 52,1,september
5729,16204440,Перенос звонка на другой день,"нет слотов, 8626",ГП 52,1,september
5730,16533889,Перенос звонка на другой день,"нет слотов, 7897",ГП 52,1,september
5731,17896313,Ни разу не звонили,NaN,ГП 52,2,september


In [101]:
final_task_predistribution = \
pd.concat([initial_task_for_today_present_assistants, additional_task_for_present_assistants]).reset_index(drop=True)

In [102]:
empty_login_patients_amount_before = final_task_predistribution[final_task_predistribution.operator_login.isna()].shape[0]
empty_login_patients_amount_before

5733

In [103]:
%%time

for mo in distribution_count.index:
    
    while final_task_predistribution[final_task_predistribution.mo == mo].groupby('operator_login') \
    .agg({'emiasid': 'count'}).min()['emiasid'] < task_amount and \
    final_task_predistribution[(final_task_predistribution.operator_login.isna()) & (final_task_predistribution.mo == mo)] \
    .shape[0] > 0:
        
        miminal_in_mo_assistant_emiasid_amount = final_task_predistribution[final_task_predistribution.mo == mo] \
        .groupby('operator_login').agg({'emiasid': 'count'}).min()['emiasid']

        minimal_in_mo_emiasid_amount_login = final_task_predistribution[final_task_predistribution.mo == mo] \
        .groupby('operator_login').agg({'emiasid': 'count'}).sort_values(by='emiasid').index[0]
        
        
        empty_login_in_mo_index = final_task_predistribution[
            (final_task_predistribution.operator_login.isna()) & (final_task_predistribution.mo == mo)
        ].index[0]
        
        final_task_predistribution = \
        final_task_predistribution.sort_values(by=['priority', 'calls_amount', 'last_call_date'])

        final_task_predistribution.loc[empty_login_in_mo_index, 'operator_login'] = minimal_in_mo_emiasid_amount_login


CPU times: total: 1min 17s
Wall time: 1min 17s


In [104]:
empty_login_patients_amount_after = final_task_predistribution[final_task_predistribution.operator_login.isna()].shape[0]
empty_login_patients_amount_after

534

In [105]:
check_table = distribution_count.merge(additional_task_for_present_assistants_by_mo,left_index=True, right_index=True) \
[['total_delta', 'emiasid_y']]

In [106]:
check_table.loc[check_table['total_delta'] > check_table['emiasid_y'], 'real_addition'] = \
check_table.loc[check_table['total_delta'] > check_table['emiasid_y'], 'emiasid_y']

check_table.loc[check_table['total_delta'] <= check_table['emiasid_y'], 'real_addition'] = \
check_table.loc[check_table['total_delta'] <= check_table['emiasid_y'], 'total_delta']

In [107]:
real_change_of_emty_login_patients_amount = check_table.real_addition.sum()
real_change_of_emty_login_patients_amount

5199.0

In [108]:
empty_login_patients_amount_before - empty_login_patients_amount_after == real_change_of_emty_login_patients_amount

True

In [109]:
final_task_distribution = final_task_predistribution[~final_task_predistribution.operator_login.isna()] \
.copy().reset_index(drop=True)

In [110]:
final_task_distribution.shape[0]

11943

In [111]:
final_task_distribution.columns

Index(['emiasid', 'status', 'old_comment', 'operator_login', 'mo', 'priority',
       'calls_amount', 'last_call_date', 'month'],
      dtype='object')

In [112]:
final_task_distribution.status.value_counts()

Ни разу не звонили                7551
Перенос звонка на другой день     2723
Будет в Москве в другом месяце    1544
Запишется самостоятельно           100
Живет не в Москве                    9
Отказ                                6
Госпитализирован                     5
Разговор состоялся                   3
Отсутствует номер                    1
Не дозвонились                       1
Name: status, dtype: int64

In [113]:
final_task_distribution.shape[0]

11943

In [114]:
final_task_distribution.columns

Index(['emiasid', 'status', 'old_comment', 'operator_login', 'mo', 'priority',
       'calls_amount', 'last_call_date', 'month'],
      dtype='object')

In [115]:
final_task_distribution.head(1)

,emiasid,status,old_comment,operator_login,mo,priority,calls_amount,last_call_date,month
0,16109737,Перенос звонка на другой день,Перезвонить,pomoshnik_pdn57,ГП 19,1,1.0,2022-09-02,september


In [116]:
second_stage_patients.head(1)

,emiasid,status,old_comment,operator_login,mo,priority,calls_amount,last_call_date,month
134,19683038,Госпитализирован,перезвонить,pomoshnik_pdn95,ДКЦ 1,4,6.0,2022-09-13,august


In [117]:
second_stage_patients = second_stage_patients[['emiasid', 'status', 'old_comment', 'operator_login', 'mo', 'priority',
       'calls_amount', 'last_call_date', 'month']]

In [118]:
tomorrow_record_patients = tomorrow_record_patients[['emiasid', 'status', 'old_comment', 'operator_login', 'mo', 'priority',
       'calls_amount', 'last_call_date', 'month']]

In [119]:
final_actual_patients_for_task_ = pd.concat([
    final_task_distribution,
    second_stage_patients,
    tomorrow_record_patients
]) \
.sort_values(['priority']).reset_index(drop=True)

final_actual_patients_for_task_.shape[0]

64526

In [ ]:
# final_actual_patients_for_task = final_actual_patients_for_task_.drop_duplicates(subset=['phone_number'], keep='first')
final_actual_patients_for_task_.shape[0]

In [ ]:
final_actual_patients_for_task_ = final_actual_patients_for_task_.drop_duplicates(subset=['emiasid'], keep='first')
final_actual_patients_for_task_.shape[0]

In [120]:
final_actual_patients_for_task_.head(3)

,emiasid,status,old_comment,operator_login,mo,priority,calls_amount,last_call_date,month
0,16109737,Перенос звонка на другой день,Перезвонить,pomoshnik_pdn57,ГП 19,1,1.0,2022-09-02,september
1,17800864,Перенос звонка на другой день,NaN,pomoshnik_pdn253,ГП 209,1,2.0,2022-09-09,september
2,162576078036,Перенос звонка на другой день,Перезвонить,pomoshnik_pdn79,ГП 64,1,2.0,2022-09-09,september


In [121]:
final_actual_patients_for_task_ = \
final_actual_patients_for_task_[['emiasid', 'status', 'old_comment', 'operator_login', 'mo', 'priority', 'calls_amount',
                                'last_call_date', 'month']]

In [122]:
final_actual_patients_for_task_['emiasid'] = final_actual_patients_for_task_['emiasid'].astype(str).str.replace('\.0', '')

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_4916\309388853.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  final_actual_patients_for_task_['emiasid'] = final_actual_patients_for_task_['emiasid'].astype(str).str.replace('\.0', '')


In [123]:
initial_assistant_to_patient_distribution_in_today_task = \
final_actual_patients_for_task_.copy()

In [124]:
initial_assistant_to_patient_distribution_in_today_task.old_comment.nunique()

3168

In [125]:
initial_assistant_to_patient_distribution_in_today_task.groupby(['mo', 'operator_login']).agg({'emiasid': 'count'}).sort_values(
    ['mo', 'emiasid']).head(20)

emiasid
mo     operator_login           
ГП 107 pomoshnik_pdn131       12
       pomoshnik_pdn130      193
       pomoshnik_pdn148      346
       pomoshnik_pdn109      424
       pomoshnik_pdn43       430
       pomoshnik_pdn185      445
       pomoshnik_pdn129      494
ГП 109 pomoshnik_pdn155       33
       pomoshnik_pdn168      243
       pomoshnik_pdn134      317
ГП 11  pomoshnik_pdn146       81
       pomoshnik_pdn184      289
       pomoshnik_pdn32       346
       pomoshnik_pdn147      554
ГП 134 pomoshnik_pdn63       315
       pomoshnik_pdn75       338
       pomoshnik_pdn93       378
       pomoshnik_pdn7        421
       pomoshnik_pdn89       492
ГП 166 Помощник 8             24

In [126]:
# initial_assistant_to_patient_distribution_in_today_task.to_excel('initial_assistant_to_patient_distribution_in_today_task.xlsx')

In [127]:
task_amount = 120

In [128]:
%%time

initial_task_for_today = pd.DataFrame()
for operator_login in set(initial_assistant_to_patient_distribution_in_today_task.operator_login):
    initial_assistant_to_patient_distribution_in_today_task_for_operator_login = \
    initial_assistant_to_patient_distribution_in_today_task[initial_assistant_to_patient_distribution_in_today_task \
                                                            .operator_login
         == operator_login].sort_values(by=['priority', 'calls_amount', 'last_call_date'])
    call_back_number_for_operator_login = initial_assistant_to_patient_distribution_in_today_task_for_operator_login[
        initial_assistant_to_patient_distribution_in_today_task_for_operator_login.status.isin({
            'Перезвонить сегодня', 'Перенос звонка на другой день'})].shape[0]

    initial_task_for_today = pd.concat(
        [initial_task_for_today, 

         
         initial_assistant_to_patient_distribution_in_today_task[
             (initial_assistant_to_patient_distribution_in_today_task.operator_login==operator_login)
         ].head(task_amount),
        
       
        ]).reset_index(drop=True)

initial_task_for_today.shape[0]

CPU times: total: 3.41 s
Wall time: 3.4 s


24290

In [129]:
initial_task_for_today \
.groupby(['mo', 'operator_login']).agg({'emiasid': 'count'}).sort_values(['mo', 'emiasid'])\
.sort_values(by='emiasid', ascending=True).head(10)

,,emiasid
mo,operator_login,
ГП 170,pomoshnik_pdn151,1
ГП 107,pomoshnik_pdn131,12
ГП 23,pomoshnik_pdn14,15
ГП 214,pomoshnik_pdn156,19
ГП 166,Помощник 8,24
ГП 52,pomoshnik_pdn103,31
ГП 109,pomoshnik_pdn155,33
ГП 36,Помощник 5,40
ГП 66,pomoshnik_pdn54,43


In [130]:
initial_task_for_today \
.groupby(['mo', 'operator_login']).agg({'emiasid': 'count'}).sort_values(['mo', 'emiasid'])\
.sort_values(by='emiasid', ascending=False).head(10)

emiasid
mo    operator_login           
ГП 45 pomoshnik_pdn299      120
ГП 66 pomoshnik_pdn249      120
ГП 64 pomoshnik_pdn105      120
      pomoshnik_pdn17       120
      pomoshnik_pdn45       120
      pomoshnik_pdn47       120
      pomoshnik_pdn79       120
      pomoshnik_pdn80       120
ГП 66 pomoshnik_pdn116      120
      pomoshnik_pdn159      120

In [131]:
rename_mo = \
{'ГБУЗ Г МОСКВЫ ГП № 170 ДЗМ': 'ГП 170', ' ГП 6 ДЗМ': 'ГП 6', 'ГБУЗ КДЦ 6 ДЗМ':'КДЦ 6', 
             'ГБУЗ "ГП № 134 ДЗМ"': 'ГП 134', 'КДЦ 4': 'КДЦ 4', 'ГБУЗ ДКЦ 1 ДЗМ': 'ДКЦ 1', 'ГБУЗ ГП 23 ДЗМ': 'ГП 23',
             'ГБУЗ ГП 109 ДЗМ': 'ГП 109', 'ГБУЗ ГП 11 ДЗМ': 'ГП 11', 'ГБУЗ ГП 64 ДЗМ': 'ГП 64', 'ГБУЗ ГП 191 ДЗМ': 'ГП 191', 
             'ГБУЗ ГП 220 ДЗМ': 'ГП 220', 'ГБУЗ "ГП № 218 ДЗМ"': 'ГП 218',  'ГБУЗ ГП 107 ДЗМ': 'ГП 107', 
             'ГБУЗ "ГП № 22 ДЗМ"': 'ГП 22', 'ГБУЗ ДЦ 5 ДЗМ': 'ДЦ 5', 'ГБУЗ ГП 210 ДЗМ': 'ГП 210', 'ГБУЗ ГП 214 ДЗМ': 'ГП 214', 
             'ГБУЗ ГП 2 ДЗМ': 'ГП 2', 'ГБУЗ КДП 121 ДЗМ': 'КДП 121', 'ГБУЗ ГП 66 ДЗМ': 'ГП 66',  'ГБУЗ ДЦ 3 ДЗМ ': 'ДЦ 3',
             'ГБУЗ ГП 36 ДЗМ': 'ГП 36',  'ГБУЗ ГП 62 ДЗМ ': 'ГП 62', 
            'ГБУЗ "ГП № 52 ДЗМ"': 'ГП 52', 'ГБУЗ "ГП 67 ДЗМ"': 'ГП 67', 'ГБУЗ ГП 68 ДЗМ': 'ГП 68', 'ГБУЗ ГП 175 ДЗМ': 'ГП 175',
'ГБУЗ ГП № 209 ДЗМ': 'ГП 209',  'ГБУЗ ГП 212 ДЗМ': 'ГП 212', 'ГБУЗ ГП 19 ДЗМ': 'ГП 19', 
'ГБУЗ ГП №166 ДЗМ': 'ГП 166', 'ГБУЗ ГП 9 ДЗМ': 'ГП 9', 'ГБУЗ ГП 69 ДЗМ': 'ГП 69',
'ГБУЗ "ГП № 3 ДЗМ"': 'ГП 3', 'ГБУЗ ГП 5 ДЗМ': 'ГП 5', 'ГБУЗ ГП 46 ДЗМ': 'ГП 46', 'ГБУЗ ГП №8 ДЗМ': 'ГП 8', 
'ГБУЗ "ГП № 45 ДЗМ"': 'ГП 45'}

In [132]:
mo_list = set(rename_mo.values())

In [133]:
# new_doctor_patients = \
# initial_assistant_to_patient_distribution_in_today_task[(initial_assistant_to_patient_distribution_in_today_task.mo.isin(mo_list)) & 
#                     (~initial_assistant_to_patient_distribution_in_today_task.doctor_fio.isin(doctors_list))].reset_index(drop=True)
# new_doctor_patients = new_doctor_patients[~new_doctor_patients.emiasid.isin(calls_data.emiasid)]

In [134]:
initial_task_for_today_present_assistants = \
initial_task_for_today[~initial_task_for_today.operator_login.isin(absent_assistants.login)]

In [135]:
initial_task_for_today[initial_task_for_today.operator_login.isin(absent_assistants.login)].shape[0]

4425

In [136]:
initial_task_for_today_present_assistants.shape[0]

19865

In [137]:
initial_task_for_today_present_assistants.groupby('operator_login').agg({'emiasid': 'count'}).sort_values('emiasid', ascending=False)

,emiasid
operator_login,
pomoshnik_pdn100,120
pomoshnik_pdn30,120
pomoshnik_pdn307,120
pomoshnik_pdn309,120
pomoshnik_pdn31,120
...,...
pomoshnik_pdn212,120
pomoshnik_pdn217,120
pomoshnik_pdn98,120


In [138]:
initial_task_for_today_absent_assistants =  \
initial_task_for_today[initial_task_for_today.operator_login.isin(absent_assistants.login)] \
[['emiasid', 'status', 'old_comment', 'mo', 'priority', 'month']]

In [139]:
additional_task_for_present_assistants = \
pd.concat([
#     new_doctor_patients,
    initial_task_for_today_absent_assistants
]).reset_index(drop=True)

In [140]:
additional_task_for_present_assistants_by_mo = \
additional_task_for_present_assistants.groupby(['mo']).agg({'emiasid': 'count'}).sort_values(by=['mo', 'emiasid'])

In [141]:
distribution_count = initial_task_for_today_present_assistants.groupby(['mo']) \
.agg({'emiasid': 'count', 'operator_login': 'nunique'}) \
.sort_values(by=['mo', 'emiasid'])

In [142]:
distribution_count['total_delta'] = distribution_count['operator_login'] * task_amount - distribution_count['emiasid']

In [143]:
distribution_count = distribution_count[distribution_count.total_delta > 0]

In [144]:
additional_task_for_present_assistants

,emiasid,status,old_comment,mo,priority,month
0,24350182,Разговор состоялся,NaN,ГП 3,4,september
1,2250333119,Не дозвонились,NaN,ГП 3,4,september
2,22769722,Не дозвонились,NaN,ГП 3,4,september
3,23843368,Не дозвонились,NaN,ГП 3,4,september
4,24283272,Не дозвонились,NaN,ГП 3,4,september
...,...,...,...,...,...,...
4420,21054808,Напомнить,NaN,ДЦ 5,5,september
4421,24462765,Напомнить,NaN,ДЦ 5,5,september
4422,2243418536,Напомнить,NaN,ДЦ 5,5,september
4423,20445972,Напомнить,NaN,ДЦ 5,5,september


In [145]:
final_task_predistribution = \
pd.concat([initial_task_for_today_present_assistants, additional_task_for_present_assistants]).reset_index(drop=True)

In [146]:
empty_login_patients_amount_before = final_task_predistribution[final_task_predistribution.operator_login.isna()].shape[0]
empty_login_patients_amount_before

4425

In [147]:
%%time

for mo in distribution_count.index:
    
    while final_task_predistribution[final_task_predistribution.mo == mo].groupby('operator_login') \
    .agg({'emiasid': 'count'}).min()['emiasid'] < task_amount and \
    final_task_predistribution[(final_task_predistribution.operator_login.isna()) & (final_task_predistribution.mo == mo)] \
    .shape[0] > 0:
        
        miminal_in_mo_assistant_emiasid_amount = final_task_predistribution[final_task_predistribution.mo == mo] \
        .groupby('operator_login').agg({'emiasid': 'count'}).min()['emiasid']

        minimal_in_mo_emiasid_amount_login = final_task_predistribution[final_task_predistribution.mo == mo] \
        .groupby('operator_login').agg({'emiasid': 'count'}).sort_values(by='emiasid').index[0]
        
        
        empty_login_in_mo_index = final_task_predistribution[
            (final_task_predistribution.operator_login.isna()) & (final_task_predistribution.mo == mo)
        ].index[0]
        
        final_task_predistribution = \
        final_task_predistribution.sort_values(by=['priority', 'calls_amount', 'last_call_date'])

        final_task_predistribution.loc[empty_login_in_mo_index, 'operator_login'] = minimal_in_mo_emiasid_amount_login


CPU times: total: 1.05 s
Wall time: 1.04 s


In [148]:
empty_login_patients_amount_after = final_task_predistribution[final_task_predistribution.operator_login.isna()].shape[0]
empty_login_patients_amount_after

4386

In [149]:
final_task_distribution = final_task_predistribution[~final_task_predistribution.operator_login.isna()] \
.copy().reset_index(drop=True)

In [151]:
final_task_distribution.status.value_counts()

Не дозвонились                    8867
Ни разу не звонили                4634
Перенос звонка на другой день     2723
Разговор состоялся                1553
Будет в Москве в другом месяце    1089
Запишется самостоятельно           672
Отказ                              225
Госпитализирован                    95
Не нравится врач                    28
Не нравится МО                       7
Живет не в Москве                    6
Напомнить                            5
Name: status, dtype: int64

In [ ]:
final_task_distribution.shape[0]

In [ ]:
final_task_distribution.columns

# ПРОВЕРКА

In [ ]:
assistants_presence[assistants_presence.absence == 1].shape[0]

In [ ]:
empty_login_patients_amount_before

In [ ]:
empty_login_patients_amount_after

In [ ]:
real_change_of_emty_login_patients_amount

In [ ]:
empty_login_patients_amount_before - empty_login_patients_amount_after == real_change_of_emty_login_patients_amount

In [ ]:
final_task_distribution.shape[0] + empty_login_patients_amount_after

In [ ]:
initial_task_for_today.shape[0] 
# + new_doctor_patients.shape[0]

In [ ]:
final_task_distribution.emiasid.isin(
    patients_current_statuses.emiasid).unique()

In [ ]:
final_task_distribution[final_task_distribution.status == 'Ни разу не звонили'].emiasid.isin(calls_data.emiasid).unique()

In [ ]:
final_task_distribution[~(final_task_distribution.status == 'Ни разу не звонили')].emiasid.isin(calls_data.emiasid).unique()

In [ ]:
call_back_today_patients.shape[0]

In [ ]:
# call_backs_overdue.shape[0]

In [ ]:
final_task_distribution = final_task_distribution.rename(columns={'login': 'operator_login'})

In [ ]:
final_task_distribution = final_task_distribution.rename(columns={'last_comment': 'old_comment'})

In [ ]:
final_task_distribution.shape[0]

In [ ]:
final_task_distribution.groupby('operator_login').agg({'emiasid': 'count'}).sort_values(by=['emiasid']).head(30)

In [ ]:
final_task_distribution['emiasid'] = final_task_distribution['emiasid']

In [ ]:
final_task_distribution[final_task_distribution.month == 'august']\
.to_excel(rf'august_task_for_today_{today_y_m_d}.xlsx', index=False)

In [ ]:
final_task_distribution[final_task_distribution.month == 'september']\
.to_excel(rf'september_task_for_today_{today_y_m_d}.xlsx', index=False)

In [ ]:
final_task_distribution.to_excel(rf'full_task_for_today_{today_y_m_d}.xlsx', index=False)

In [ ]:
assistants_presence